# Week 2 - Preprocessing, part 2

# 1. Lesson: None

# 2. Weekly graph question

The Storytelling With Data book mentions planning on a "Who, What, and How" for your data story.  Write down a possible Who, What, and How for your data, using the ideas in the book.

*DATASET*
Flight delay dataset

*WHO* 
Airline Operations Leaders who want to improve the flight experience for customers and reduce operational costs.

*WHAT*
They want to know which factors contribute flight delays in order to optimize the deployment of resources and communication to customers.

*HOW*
The Flight Delay dataset can help idenitfy delay factors by route, weather, temperature, airline, etc.

# 3. Homework - work with your own data

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt


This week, you will do the same types of exercises as last week, but you should use your chosen datasets that someone in your class found last semester. (They likely will not be the particular datasets that you found yourself.)

### Here are some types of analysis you can do  Use Google, documentation, and ChatGPT to help you:

- Summarize the datasets using info() and describe()

- Are there any duplicate rows?

- Are there any duplicate values in a given column (when this would be inappropriate?)

- What are the mean, median, and mode of each column?

- Are there any missing or null values?

    - Do you want to fill in the missing value with a mean value?  A value of your choice?  Remove that row?

- Identify any other inconsistent data (e.g. someone seems to be taking an action before they are born.)

- Encode any categorical variables (e.g. with one-hot encoding.)

### Conclusions:

- Are the data usable?  If not, find some new data!

- Do you need to modify or correct the data in some way?

- Is there any class imbalance?  (Categories that have many more items than other categories).

# Flight Delay Dataset Examination

## Conclusion

- Overall this is a strong dataset with a lot of detail around flights delays and potential causal columns for weather and temperature.
- There are some ID columns that need to be mapped if possible.

### Class Imbalance
- THere doesn't appear to be a significant amount of class imbalance.
- Small hub airport origin/destination is 15%
- temp_ninfty_n10 is 0.86%
- temp_n10_0 is 5.75%
- temp_40_infty is 0.33%



## Observations

### Info on Dataset
- 1.2m rows, 61 columns
- No rows are all null
- There are almost no null values. The only exception is arrdelay which has ~3k null rows.
- There are zero duplicate rows.

### Stats on each column
- depdelay has a min value of -866
- There's a temperature of -42 celsius
- The rest of the columns look strong and consistent.



## Code Section: Flight Delay Dataset

In [ ]:
df = pd.read_csv("DelayData.csv", delimiter=",", low_memory=False)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('[^a-zA-Z0-9_]', '', regex=True).str.replace('__','_')
df_clean = df.dropna(axis=1, how='all')

In [ ]:
pd.set_option('display.max_info_columns', 200)
df_clean.info()

In [ ]:
df_clean.describe()

In [ ]:
# Check for duplicate rows
duplicate_rows = df_clean.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicate_rows}")

In [ ]:
#Analyze stats for each column

# Show all rows and columns in output
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)

# Prevent truncation of wide columns
pd.set_option('display.width', 150)
pd.set_option('display.max_colwidth', 200)


# Select numeric columns excluding those ending with '_id'
numeric_df = df_clean.select_dtypes(include=['number'])
numeric_df = numeric_df.loc[:, ~numeric_df.columns.str.endswith('_id')]

# Calculate statistics for each numeric column
mean_values = numeric_df.mean()
median_values = numeric_df.median()
mode_values = numeric_df.mode().iloc[0]
min_values = numeric_df.min()
max_values = numeric_df.max()
variance_values = numeric_df.var()
std_dev_values = numeric_df.std()

# Create a DataFrame to summarize the statistics
summary_df = pd.DataFrame({
    'Column Name': numeric_df.columns,
    'Mean': mean_values,
    'Median': median_values,
    'Mode': mode_values,
    'Min': min_values,
    'Max': max_values,
    'Variance': variance_values,
    'Standard Deviation': std_dev_values
})

# Set 'Column Name' as the index
summary_df.set_index('Column Name', inplace=True)

# Display the summary DataFrame
print(summary_df)



In [ ]:
# Perform one-hot encoding on all object-type (categorical) columns
encode_cols = ['origin','dest', 'uniquecarrier', 'originstate', 'origincityname']
df_encoded = pd.get_dummies(df_clean, columns=encode_cols)

In [ ]:
# Function to identify binary columns and measure class imbalance in percentages
def measure_binary_class_imbalance(df):
    binary_columns = [col for col in df.columns if df[col].nunique() == 2]
    imbalance_info = {}

    for column in binary_columns:
        value_counts = df[column].value_counts()
        total = value_counts.sum()
        percentage_distribution = (value_counts / total) * 100
        imbalance_info[column] = percentage_distribution

    return imbalance_info

# Measure class imbalance in binary columns
binary_imbalance_info = measure_binary_class_imbalance(df_clean)

# Print imbalance information
for column, value_counts in binary_imbalance_info.items():
    print(f"Column: {column}")
    print(value_counts.round(2))  # Rounded for readability
    print()


In [ ]:
# Function to identify class imbalance in categorical columns
def identify_class_imbalance(df):
    categorical_columns = df.select_dtypes(include=['object']).columns
    imbalance_info = {}

    for column in categorical_columns:
        if df[column].nunique() <= 100:  # Exclude columns with too many unique values
            value_counts = df[column].value_counts()
            imbalance_info[column] = value_counts

    return imbalance_info

# Identify class imbalance
imbalance_info = identify_class_imbalance(df_clean)

# Visualize class imbalance
for column, value_counts in imbalance_info.items():
    plt.figure(figsize=(10, 6))
    value_counts.plot(kind='bar')
    plt.title(f'Class Imbalance in {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.show()


# USDOT On Time Flight Reporting Dataset Examination

## Conclusion

- Overall this looks like a strong dataset with lots of opportunity to analyze and train. It would be ideal to download additional months of data for completeness.

### Class Imbalance
- There are a high proportion of flights through California, Florida, and Texas. This makes sense given the population and tourism in these states.
- Only 1.5% of flights were cancelled.
- Only 0.2% of flights were diverted.

## Observations

### INFO() on the base dataset
- 109 columns, 504,884 rows
- All the DIV4 and DIV5 columns are null
- Approx 7k rows are missing key data around departure delays.

### Analyzing DESCRIBE()
- There are lots of ID columns that require lookup files
- This dataset only includes data for February 2025. It would be useful to get additional months of data.

#### Mean, Median, Model values:
- The output is quite large and is stored in summary_df
                      
#### Missing values:
- There's missing values on first, second, and third diversion columns. This makes sense if a flight wasn't diverted

#### Inconsistent Data
- There are no duplicate rows
- There are rows with negative values for delay time. I'm interpretting this as flights leaving or arriving early.
- There are 3 columns which capture origin state and another 3 that capture destination state. These can be collapse.

## Code Section: USDOT On Time Flight Dataset

In [ ]:
# Load the On Time Flight Dataset
df = pd.read_csv("T_ONTIME_REPORTING.csv", delimiter=",", low_memory=False)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('[^a-zA-Z0-9_]', '', regex=True).str.replace('__','_')
df_clean = df.dropna(axis=1, how='all')

In [ ]:
pd.set_option('display.max_info_columns', 200)
df_clean.info()

In [ ]:
df_clean.describe()

In [ ]:
# Check for duplicate rows
duplicate_rows = df_clean.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicate_rows}")

In [ ]:
#Analyze stats for each column

# Show all rows and columns in output
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)

# Prevent truncation of wide columns
pd.set_option('display.width', 150)
pd.set_option('display.max_colwidth', 200)


# Select numeric columns excluding those ending with '_id'
numeric_df = df_clean.select_dtypes(include=['number'])
numeric_df = numeric_df.loc[:, ~numeric_df.columns.str.endswith('_id')]

# Calculate statistics for each numeric column
mean_values = numeric_df.mean()
median_values = numeric_df.median()
mode_values = numeric_df.mode().iloc[0]
min_values = numeric_df.min()
max_values = numeric_df.max()
variance_values = numeric_df.var()
std_dev_values = numeric_df.std()

# Create a DataFrame to summarize the statistics
summary_df = pd.DataFrame({
    'Column Name': numeric_df.columns,
    'Mean': mean_values,
    'Median': median_values,
    'Mode': mode_values,
    'Min': min_values,
    'Max': max_values,
    'Variance': variance_values,
    'Standard Deviation': std_dev_values
})

# Set 'Column Name' as the index
summary_df.set_index('Column Name', inplace=True)

# Display the summary DataFrame
print(summary_df)



In [ ]:
# Perform one-hot encoding on all object-type (categorical) columns
encode_cols = ['op_unique_carrier','op_carrier','origin','origin_city_name','origin_state_abr','dest','dest_city_name','dest_state_abr','dep_time_blk','arr_time_blk','cancellation_code']
df_encoded = pd.get_dummies(df_clean, columns=encode_cols)

In [ ]:

# Function to identify class imbalance in categorical columns
def identify_class_imbalance(df):
    categorical_columns = df.select_dtypes(include=['object']).columns
    imbalance_info = {}

    for column in categorical_columns:
        if df[column].nunique() <= 100:  # Exclude columns with too many unique values
            value_counts = df[column].value_counts()
            imbalance_info[column] = value_counts

    return imbalance_info

# Identify class imbalance
imbalance_info = identify_class_imbalance(df_clean)

# Visualize class imbalance
for column, value_counts in imbalance_info.items():
    plt.figure(figsize=(10, 6))
    value_counts.plot(kind='bar')
    plt.title(f'Class Imbalance in {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.show()


In [ ]:
# Function to identify binary columns and measure class imbalance in percentages
def measure_binary_class_imbalance(df):
    binary_columns = [col for col in df.columns if df[col].nunique() == 2]
    imbalance_info = {}

    for column in binary_columns:
        value_counts = df[column].value_counts()
        total = value_counts.sum()
        percentage_distribution = (value_counts / total) * 100
        imbalance_info[column] = percentage_distribution

    return imbalance_info

# Measure class imbalance in binary columns
binary_imbalance_info = measure_binary_class_imbalance(df_clean)

# Print imbalance information
for column, value_counts in binary_imbalance_info.items():
    print(f"Column: {column}")
    print(value_counts.round(2))  # Rounded for readability
    print()


# Priceline Flight Dataset Examination

## Conclusion
- After several data cleansing steps the datasets appears functional. However, the data lack significant utility as it only captures pricing data for 2 days in April 2025 and lacks significant breadth or depth.
- None of the columns show signifcant class imbalance. Though 25% of the records list "Multiple" airlines.
- Additional features that may be interesting: day of week, day of month, month of year, hour of day, region/continent, departure country, arrival country, domestic/international flight.

## Observations
### Data Quality
- There are several empty columns. The CSV appears to be corrupt. We will remove the bad columns.
- Date and Time columns are text.
- Price column is an object, not numeric.
- There are some rows of bad data. Dollar sign in the price column. Number of Stops = Express Deal. Travel Time = Save $39. etc. These rows will be removed.
### INFO() on the base dataset
- 15 columns, 2,459 rows
- 2 rows missing an airline name
- 296 rows missing a departure airport
- 3 rows missing a ticket price
- 299 rows missing an arrival airport (maybe they never left?)
- Several rows don't have 2nd or 3rd stoppage values. These are most likely flights that were one leg
### Analyzing the clean dataset
- 57 unique airlines
- Travel time rangs from ~2.66 hours to 82 hours
- Wait times range from 39 minutes to 390 minutes
- Ticker prices ranges from $135 to $8000
- 28 duplicate rows

#### Mean values:
- travel_time_minutes      1551.063440
- 1st_stop_wait_minutes     510.922116
- 2nd_stop_wait_minutes     426.108563
- 3rd_stop_wait_minutes     356.764706
- stops                       1.294429
- ticket_price_usd         1316.544603

#### Median values:
- travel_time_minutes      1430.0
- 1st_stop_wait_minutes     424.0
- 2nd_stop_wait_minutes     260.0
- 3rd_stop_wait_minutes     200.0
- stops                       1.0
- ticket_price_usd         1128.0

#### Mode values:
- travel_time_minutes       965.0
- 1st_stop_wait_minutes     130.0
- 2nd_stop_wait_minutes     240.0
- 3rd_stop_wait_minutes     200.0
- stops                       1.0
- ticket_price_usd         1508.0

#### Missing values:
- airline_name                2 (These should dropped)
- depreture_airport         296 (These need to be investigated and potentially dropped)
- 1st_stoppage               59 (It is valid that a flight may not have a stopover)
- 2nd_stoppage             1803 (It is valid that a flight may not have a stopover)
- 3rd_stoppage             2442 (It is valid that a flight may not have a stopover)
- destination_airport       293 (These need to be investigated and potentially dropped)
- travel_time_minutes         0
- 1st_stop_wait_minutes      58 (It is valid that a flight may not have a stopover) 
- 2nd_stop_wait_minutes    1805 (It is valid that a flight may not have a stopover)
- 3rd_stop_wait_minutes    2442 (It is valid that a flight may not have a stopover)
- stops                       0
- ticket_price_usd            4 (Impute using mean or regression)
- arrival_datetime          164 (These need to be investigated and potentially dropped)
- departure_time_24hr         0

#### Inconsistent Data
- It doesn't seem reasonable that a flight would be missing a departure or arrival airport.
- All wait times, stops, and prices seem valid.

## Code Section: Priceline Dataset

In [ ]:
# Load the Flight Dataset
df_src = pd.read_csv("flight.csv", delimiter=",", low_memory=False)

def flight_data_cleaner(df):
    # Remove the junk columns
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    # Clean up the column names
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('[^a-zA-Z0-9_]', '', regex=True).str.replace('__','_')
    return df

def convert_to_minutes(time_str):
    if pd.isna(time_str):
        return np.nan
    if isinstance(time_str, float):
        return np.nan
    try:
        total_minutes = 0
        if 'h' in time_str:
            hours, minutes = time_str.split('h ')
            total_minutes += int(hours) * 60
            total_minutes += int(minutes.replace('m', ''))
        else:
            total_minutes += int(time_str.replace('m', ''))
        return total_minutes
    except:
        return None

df_src = flight_data_cleaner(df_src)
df_clean = df_src.copy()
df_clean['travel_time_minutes'] = df_clean['travel_time'].apply(convert_to_minutes)
df_clean['1st_stop_wait_minutes'] = df_clean['1st_stoppage_waiting_hour'].apply(convert_to_minutes)
df_clean['2nd_stop_wait_minutes'] = df_clean['2nd_stoppagewaiting_time'].apply(convert_to_minutes)
df_clean['3rd_stop_wait_minutes'] = df_clean['3rd_stoppage_waiting_time'].apply(convert_to_minutes)


# Map the stops column
stops_mapping = {
    'Nonstop': 0,
    '1 Stop': 1,
    '2 Stops': 2,
    '3 Stops': 3
}
df_clean['stops'] = df_clean['number_of_stoppage'].map(stops_mapping)

# Replace any value containing a dollar sign with None
df_clean['ticket_prizedoller'] = df_clean['ticket_prizedoller'].apply(lambda x: None if '$' in str(x) or 'Alaska' in str(x) else x)
# Convert the entire column to float
df_clean['ticket_price_usd'] = df_clean['ticket_prizedoller'].astype(float)


def clean_and_combine_datetime(df, date_col, time_col):
    def parse_datetime(date_str, time_str):
        try:
            # Strip leading/trailing spaces and prefixes
            if isinstance(date_str, str):
                date_str = date_str.strip()
                if 'Arrives:' in date_str:
                    date_str = date_str.split('Arrives: ')[-1]
                date_str = date_str.split(', ')[-1]
            else:
                return None
            
            if isinstance(time_str, str):
                time_str = time_str.strip().upper().replace('A', 'AM').replace('P', 'PM')
            else:
                return None
            
            # Add current year to date string
            current_year = datetime.now().year
            date_str = f"{date_str} {current_year}"
            
            # Combine date and time strings
            datetime_str = f"{date_str} {time_str}"
            
            # Parse combined datetime string
            return datetime.strptime(datetime_str, '%b %d %Y %I:%M%p')
        except ValueError:
            return None
    
    # Apply the parsing function to the DataFrame
    df['arrival_datetime'] = df.apply(lambda row: parse_datetime(row[date_col], row[time_col]), axis=1)
    
    return df

# Clean and combine datetime
df_clean = clean_and_combine_datetime(df, 'arrival_date', 'arrival_time')

def convert_to_24_hour(time_str):
    try:
        # Check if the value is a string
        if isinstance(time_str, str):
            # Normalize lowercase 'a'/'p' to 'AM'/'PM'
            time_str = time_str.replace('a', 'AM').replace('p', 'PM')
            # Convert to 24-hour format
            return datetime.strptime(time_str, '%I:%M%p').strftime('%H:%M')
        else:
            return None
    except (ValueError, TypeError):
        # Return None for invalid or missing values
        return None

# Apply the conversion function to the 'depreture_time' column
df_clean['departure_time_24hr'] = df_clean['depreture_time'].apply(convert_to_24_hour)
df_clean.rename(columns={'depreture_airport': 'departure_airport'}, inplace=True)


#Remove all the junk columns
junk_cols = {'travel_time', 'number_of_stoppage','depreture_time','ticket_prizedoller','1st_stoppage_waiting_hour','2nd_stoppagewaiting_time','3rd_stoppage_waiting_time','arrival_date','arrival_time'}

df_clean = df_clean.drop(columns=junk_cols)


In [ ]:
print(df_src.info())

In [ ]:
df_clean.describe()
df_clean.info()

In [ ]:
# Check for duplicate rows
duplicate_rows = df_clean.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicate_rows}")

In [ ]:
# Filter numeric columns
numeric_df = df_clean.select_dtypes(include=['number'])

# Calculate mean, median, and mode of each numeric column
mean_values = numeric_df.mean()
median_values = numeric_df.median()
mode_values = numeric_df.mode().iloc[0]

print("\nMean values:")
print(mean_values)
print("\nMedian values:")
print(median_values)
print("\nMode values:")
print(mode_values)

In [ ]:
# Check for missing or null values
missing_values = df_clean.isnull().sum()
print("\nMissing values:")
print(missing_values)

In [ ]:
# Fill missing values with mean
df['ticket_price_usd'] = df['ticket_price_usd'].fillna(df_clean['ticket_price_usd'].mean())

In [ ]:
# Perform one-hot encoding on all object-type (categorical) columns
encode_cols = ['airline_name','departure_airport', '1st_stoppage','2nd_stoppage','3rd_stoppage','destination_airport']
df_encoded = pd.get_dummies(df_clean, columns=encode_cols)

In [ ]:
# Function to identify class imbalance in categorical columns
def identify_class_imbalance(df):
    categorical_columns = df.select_dtypes(include=['object']).columns
    imbalance_info = {}
    
    for column in categorical_columns:
        value_counts = df[column].value_counts()
        imbalance_info[column] = value_counts
    
    return imbalance_info

# Identify class imbalance
imbalance_info = identify_class_imbalance(df_clean)

# Print imbalance information
for column, value_counts in imbalance_info.items():
    print(f"Column: {column}")
    print(value_counts)
    print()

# Visualize class imbalance
for column, value_counts in imbalance_info.items():
    plt.figure(figsize=(10, 6))
    value_counts.plot(kind='bar')
    plt.title(f'Class Imbalance in {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.show()



# 4. Storytelling With Data graph

Just like last week: choose any graph in the Introduction of Storytelling With Data. Use matplotlib to reproduce it in a rough way. I don't expect you to spend an enormous amount of time on this; I understand that you likely will not have time to re-create every feature of the graph. However, if you're excited about learning to use matplotlib, this is a good way to do that. You don't have to duplicate the exact values on the graph; just the same rough shape will be enough.  If you don't feel comfortable using matplotlib yet, do the best you can and write down what you tried or what Google searches you did to find the answers.

In [ ]:
import matplotlib.pyplot as plt

